In [0]:
import logging

In [0]:

def setup_logger():
    logger = logging.getLogger("dim_customers_logger")
    logger.setLevel(logging.INFO)
    if not logger.handlers:
        handler = logging.StreamHandler()
        formatter = logging.Formatter('%(asctime)s %(levelname)s %(message)s')
        handler.setFormatter(formatter)
        logger.addHandler(handler)
    return logger

In [0]:


def create_dim_customers_view():
    logger = setup_logger()
    logger.info("Starting creation of gold.dim_customers view.")
    spark.sql('''
    CREATE OR REPLACE VIEW dwh.gold.dim_customers AS
    SELECT
        ROW_NUMBER() OVER (ORDER BY ci.cst_id) AS customer_key,  -- Surrogate key
        ci.cst_id AS customer_id,
        ci.cst_key AS customer_number,
        ci.cst_firstname AS first_name,
        ci.cst_lastname AS last_name,
        la.cntry AS country,
        ci.cst_marital_status AS marital_status,
        CASE 
            WHEN ci.cst_gndr != 'n/a' THEN ci.cst_gndr  -- CRM is the primary source for gender
            ELSE COALESCE(ca.gen, 'n/a')  -- Fallback to ERP data
        END AS gender,
        ca.bdate AS birthdate,
        ci.cst_create_date AS create_date
    FROM dwh.silver.crm_cust_info ci
    LEFT JOIN dwh.silver.erp_cust_az12 ca ON ci.cst_key = ca.cid
    LEFT JOIN dwh.silver.erp_loc_a101 la ON ci.cst_key = la.cid;
    ''')
    logger.info("gold.dim_customers view created successfully.")
    


In [0]:
def create_dim_products_view():
    logger = setup_logger()
    logger.info("Starting creation of gold.dim_products view.")
    spark.sql('''
    CREATE OR REPLACE VIEW dwh.gold.dim_products AS
    SELECT
        ROW_NUMBER() OVER (ORDER BY pn.prd_start_dt, pn.prd_key) AS product_key,  -- Surrogate key
        pn.prd_id AS product_id,
        pn.prd_key AS product_number,
        pn.prd_nm AS product_name,
        pn.cat_id AS category_id,
        pc.cat AS category,
        pc.subcat AS subcategory,
        pc.maintenance AS maintenance,
        pn.prd_cost AS cost,
        pn.prd_line AS product_line,
        pn.prd_start_dt AS start_date
    FROM dwh.silver.crm_prd_info pn
    LEFT JOIN dwh.silver.erp_px_cat_g1v2 pc ON pn.cat_id = pc.id
    WHERE pn.prd_end_dt IS NULL;  -- Filter out historical data
    ''')
    logger.info("gold.dim_products view created successfully.")

In [0]:
def create_fact_sales_view():
    logger = setup_logger()
    logger.info("Starting creation of gold.fact_sales view.")
    spark.sql('''
    CREATE OR REPLACE VIEW dwh.gold.fact_sales AS
    SELECT
        sd.sls_ord_num AS order_number,
        pr.product_key AS product_key,
        cu.customer_key AS customer_key,
        sd.sls_order_dt AS order_date,
        sd.sls_ship_dt AS shipping_date,
        sd.sls_due_dt AS due_date,
        sd.sls_sales AS sales_amount,
        sd.sls_quantity AS quantity,
        sd.sls_price AS price
    FROM dwh.silver.crm_sales_details sd
    LEFT JOIN dwh.gold.dim_products pr ON sd.sls_prd_key = pr.product_number
    LEFT JOIN dwh.gold.dim_customers cu ON sd.sls_cust_id = cu.customer_id;
    ''')
    logger.info("gold.fact_sales view created successfully.")

In [0]:
create_dim_customers_view()

2025-07-29 17:19:55,936 INFO Starting creation of gold.dim_customers view.
2025-07-29 17:19:56,407 INFO gold.dim_customers view created successfully.


In [0]:
create_dim_products_view()

2025-07-29 17:25:25,557 INFO Starting creation of gold.dim_products view.
2025-07-29 17:25:26,282 INFO gold.dim_products view created successfully.


In [0]:
create_fact_sales_view()

2025-07-29 17:25:51,418 INFO Starting creation of gold.fact_sales view.
2025-07-29 17:25:52,055 INFO gold.fact_sales view created successfully.
